Include the script in the response?



In [ ]:
var getScriptsAndStyles = (content, additional = '0.*.js') => {    var output = content.match(/<style.*?\/style>/ig) || '';    var scripts = ``;    var m, re = (/<link.*?>/ig);    while (m = re.exec(content)) {        var src;        if ((src = (/href="(.*?)"/ig).exec(m[0])) != null && src[1].indexOf('google') == -1) {            scripts += '<style>' + fs.readFileSync(path.join(project, 'public', src[1])) + '</style>';        } else {            scripts += m[0];        }    }    var m, re = (/<script.*?\/script>/ig);    while (m = re.exec(content)) {        var src;        if ((src = (/src="(.*?)"/ig).exec(m[0])) != null) {            scripts += '<script defer>'                + fs.readFileSync(path.join(project, 'public', src[1]))                + '</script>';        } else {            scripts += m[0];        }    }    return output + scripts;};module.exports = getScriptsAndStyles;getScriptsAndStyles;

Display angular modules in jupyter?



In [ ]:
var importer = require('../Core');var vm = require('vm');var fs = require('fs');var path = require('path');var renderer;var displAngular = (project, url) => {    var innerHtml = '';    var scripts = '';    process.chdir(path.join(project));    module.paths.unshift(path.join(project, 'node_modules'));    require('module').Module._initPaths();    process.argv = [(process.argv0 = path.join(project, '.server', 'render-service.js'))];    return new Promise((resolve, reject) => {        try {            var rendererCode = fs.readFileSync(path.join(project, '.server', 'render-service.js')).toString();            importer.runInNewContext(                rendererCode + '\n\n;(global.module.exports=global.renderer);',                {                    __filename: path.join(project, '.server', 'render-service.js')                },                {                    displayErrors: true                }            );        } catch (e) {            return reject(e);        }        renderer(url, (err, html) => {            if (err) {                return reject(err);            }            var content = fs.readFileSync(path.join(project, 'public', 'index.html')).toString();            scripts = getScriptsAndStyles(content);            innerHtml = html;            return resolve(innerHtml);        });    })        .then(() => importer.import('list files in project'))        .then(listInProject => listInProject(project, '**/public/assets/0.*.js'))        .then(paths => paths.map(p => '<script defer>' + fs.readFileSync(p) + '</script>')[0])        // TODO convert to using the same type of template system as angular-cli now that typescript is becoming more relevant here.        .then(chunks => ({            html: `<div id="container" style="display:block; width:100%;position:relative;"><iframe id="cell1" style="position:absolute;top:0;right:0;bottom:0;left:0;width:100%;height:100%;border:0;"></iframe></div><script defer>var ifrm = document.getElementById('cell1');var doc = ifrm.contentWindow || ifrm.contentDocument.document || ifrm.contentDocument;doc.document.open();doc.INITIAL_PATH=` + JSON.stringify('' + url) + `;doc.SOCKIFY_SERVER='http://local:8000';doc.document.write(decodeURI(` + JSON.stringify(encodeURI(innerHtml)) + `));var contHeight;contHeight = setInterval(function () {    if(document.getElementById("container") == null) {        console.log('Resizing canceled');        clearInterval(contHeight);    } else {        doc.document.getElementsByTagName('app-root')[0].style.display = 'block';        document.getElementById("container").style.height = (doc.document.getElementsByTagName('app-root')[0].offsetHeight + 40) + 'px';    }}, 500);</script>`, scripts: scripts, chunks: chunks        }));};module.exports = displAngular;displAngular;

In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;var project = PROFILE_PATH + '/universal';if (!fs.existsSync(project)) {    fs.mkdirSync(project);}var rateLimit = 900000;var outputSlowly = (r, out) => {    var counter = 0;    return new Promise(resolve => {        var interval;        var scriptPieces = Math.ceil(r.scripts.length / rateLimit);        console.log(scriptPieces + ' pieces');        interval = setInterval(() => {            if (counter < scriptPieces) {                out(r.scripts.substr(counter * rateLimit, rateLimit), counter, scriptPieces);            } else {                clearInterval(interval);                resolve(r.compress);            }            counter++;        }, 1100);    });};var compress = (code) => {    console.log('Compressing...');    return importer.import('lzw compress')        .then(lzw => ({compress: r[0].code, scripts: lzw.compressToBase64(code.scripts + code.chunks)}));};$$.async();try {    displAngular(project, '/search')        .then(r => {            $$.html(r.html, true);            return compress(r);        })        .then(r => {            console.log('App will be available in ');            return outputSlowly(r, (c, i, t) => {                console.log((t - i) + ' - ' + c.substr(0, 10));                $$.html(`<script>if(!doc.app) { doc.app=''; }doc.app+=` + JSON.stringify(c) + `;</script>`, true);            });        })        .then(compress => $$.html(`<script>eval(decodeURI(` + JSON.stringify(encodeURI(compress)) + `));doc.document.write(LZString.decompressFromBase64(doc.app));doc.document.close();</script>`, true))        .then(() => importer.import('search notebook provider'))        .then(searchProvider => searchProvider())        .then(() => $$.sendResult())        .catch(e => $$.sendError(e));} catch (e) {    $$.sendError(e);}

In [ ]:
// TODO: move this to web extension code?var importer = require('../Core');var fs = require('fs');var https = require('https');var app = require('express')();var httpsOptions = {    key: fs.readFileSync('../Utilities/.ca/intermediate/private/localhost.key.pem'),    cert: fs.readFileSync('../Utilities/.ca/intermediate/certs/localhost.cert.pem'),    passphrase: 'x',    //requestCert: true,    rejectUnauthorized: false};var secureServer = require('https').createServer(httpsOptions, app);var trustedCa = ['../Utilities/ca/intermediate/certs/localhost.cert.pem'];https.globalAgent.options.ca = [];for (const ca of trustedCa) {    https.globalAgent.options.ca.push(httpsOptions.cert);}$$.async();var searchProvider, sockify, automock, seleniumServer;importer.import('mock properties rewire')    .then(automock => {        return importer.import([            'search notebook provider',            'socket.io server',            'selenium http server'        ], {automock});    })    .then(r => {        sockify = r[1];        searchProvider = r[0];        seleniumServer = r[2];        var listener = secureServer.listen(8000);        sockify.sockifyServer(listener);        sockify.sockifyRequire({            prototype: {                search: function () {                },                results: function () {                }            }        }, 'SearchService');        sockify.sockifyRequire({            prototype: {                chrome: function () {                }            }        }, 'BrowserService');        return searchProvider();    })    .then(() => seleniumServer())    .catch(e => $$.sendError(e));// var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;// var keyChain = path.join(PROFILE_PATH, '/Library/Keychains/login.keychain');// security add-trusted-cert -d -r trustAsRoot -k ../Library/Keychains/login.keychain ../server.crt/// /* TODO: integrate with native messaging?{  "name": "com.my_company.my_application",  "description": "My Application",  "path": "C:\\Program Files\\My Application\\chrome_native_messaging_host.exe",  "type": "stdio",  "allowed_origins": [    "chrome-extension://knldjmfmopnpolahpmmgbagdohdnhkik/"  ]}*/

Requiring SearchService
{ SearchService: { prototype: { search: [Function], results: [Function] } } }
Requiring BrowserService
{ SearchService: { prototype: { search: [Function], results: [Function] } },
  BrowserService: { prototype: { chrome: [Function] } } }
Search handler


(node:14505) DeprecationWarning: sprintf() will be removed in the next major release, use the sprintf-js package instead.


Client connection
Client connection
Handler for SearchService
Client connection
Handler for BrowserService
Client connection
Client connection
Client connection
Handler for BrowserService


Error: not implemented!
    at router.get (evalmachine.<anonymous>:10:15)
    at Layer.handle [as handle_request] (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/layer.js:95:5)
    at next (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/route.js:137:13)
    at Route.dispatch (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/route.js:112:3)
    at Layer.handle [as handle_request] (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/layer.js:95:5)
    at /Users/briancullinan/jupytangular2/node_modules/express/lib/router/index.js:281:22
    at param (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/index.js:354:14)
    at param (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/index.js:365:14)
    at Function.process_params (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/index.js:410:3)
    at next (/Users/briancullinan/jupytangular2/node_modules/express/lib/rou

TypeError: Cannot read property 'objectId' of undefined
    at promisifyChrome.then.r (evalmachine.<anonymous>:22:35)
    at <anonymous>
    at process._tickCallback (internal/process/next_tick.js:188:7)


(node:14505) UnhandledPromiseRejectionWarning: Unhandled promise rejection (rejection id: 1): Either tab id or extension id must be specified.
(node:14505) [DEP0018] DeprecationWarning: Unhandled promise rejections are deprecated. In the future, promise rejections that are not handled will terminate the Node.js process with a non-zero exit code.


TypeError: Cannot read property 'objectId' of undefined
    at promisifyChrome.then.r (evalmachine.<anonymous>:22:35)
    at <anonymous>
    at process._tickCallback (internal/process/next_tick.js:188:7)
Either tab id or extension id must be specified.
Either tab id or extension id must be specified.
Client connection
Client connection
Handler for BrowserService
Client connection
Handler for BrowserService
Client connection
Handler for BrowserService
Client connection
Handler for BrowserService
Client connection
Handler for BrowserService
Client connection
Handler for BrowserService
Client connection
Handler for BrowserService
Client connection
Handler for BrowserService
Client connection
Handler for BrowserService


Error: not implemented!
    at router.get (evalmachine.<anonymous>:10:15)
    at Layer.handle [as handle_request] (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/layer.js:95:5)
    at next (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/route.js:137:13)
    at Route.dispatch (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/route.js:112:3)
    at Layer.handle [as handle_request] (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/layer.js:95:5)
    at /Users/briancullinan/jupytangular2/node_modules/express/lib/router/index.js:281:22
    at param (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/index.js:354:14)
    at param (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/index.js:365:14)
    at Function.process_params (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/index.js:410:3)
    at next (/Users/briancullinan/jupytangular2/node_modules/express/lib/rou

Client connection
Handler for BrowserService
Client connection
Handler for BrowserService
Client connection
Handler for BrowserService
Client connection
Handler for BrowserService
Client connection
Handler for BrowserService
Client connection


In [ ]:
var automock = require('../../universal/src/imports/automock.js');automock.mockValue({}, {    stubCreator: function (name) {        console.log(name);        return function () {        }    }})